In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [3]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [5]:
df_raw.shape

(37224, 14)

In [10]:
df_an = df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [11]:
df_an.shape

(6000, 14)

In [12]:
df_an = RSI(df_an)

In [15]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
5995,2021-12-30 19:00:00+00:00,721,155.312,155.444,155.274,155.444,155.296,155.432,155.260,155.432,155.328,155.458,155.289,155.457,61.286263
5996,2021-12-30 20:00:00+00:00,1001,155.442,155.470,155.289,155.312,155.429,155.458,155.275,155.298,155.456,155.484,155.303,155.326,55.345932
5997,2021-12-30 21:00:00+00:00,638,155.317,155.382,155.286,155.382,155.301,155.362,155.270,155.362,155.333,155.404,155.300,155.402,57.688104
5998,2021-12-30 22:00:00+00:00,623,155.353,155.381,155.282,155.329,155.278,155.315,155.207,155.254,155.428,155.456,155.357,155.404,55.322082
5999,2021-12-30 23:00:00+00:00,419,155.336,155.360,155.284,155.310,155.262,155.329,155.262,155.294,155.411,155.418,155.299,155.327,54.459758


In [16]:
df_an = apply_patterns(df_an)

In [17]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [18]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [25]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [26]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [27]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2021-01-27 05:00:00+00:00,142.404,142.499,142.399,142.490,142.503,142.478,True,1,141.854528,67.293687
1,2021-01-27 06:00:00+00:00,142.488,142.496,142.402,142.476,142.490,142.463,False,-1,141.861668,66.115829
2,2021-01-27 07:00:00+00:00,142.474,142.663,142.426,142.596,142.608,142.583,True,1,141.870092,70.828951
3,2021-01-27 08:00:00+00:00,142.594,142.626,142.365,142.512,142.523,142.502,False,-1,141.877445,64.106935
4,2021-01-27 09:00:00+00:00,142.510,142.612,142.411,142.555,142.565,142.545,False,1,141.885195,65.891472


In [28]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

# you can probaly change if statements into AND statements
def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE

In [29]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [30]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    5252
 1     337
-1     212
Name: count, dtype: int64

In [31]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [32]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [35]:
df_slim[df_slim.SIGNAL==SELL].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
902,2021-03-19 18:00:00+00:00,151.132,151.190,150.994,151.026,151.039,151.014,True,-1,151.324994,36.397838,-1,150.8670,151.132
909,2021-03-22 01:00:00+00:00,150.612,150.625,150.413,150.552,150.562,150.543,True,-1,151.281219,34.305156,-1,150.4620,150.612
936,2021-03-23 04:00:00+00:00,150.530,150.586,150.418,150.425,150.437,150.413,True,-1,151.133297,37.972246,-1,150.2675,150.530
940,2021-03-23 08:00:00+00:00,150.578,150.578,150.142,150.199,150.210,150.188,True,-1,151.104443,34.619073,-1,149.6305,150.578
951,2021-03-23 19:00:00+00:00,149.606,149.635,149.280,149.296,149.307,149.284,True,-1,150.950248,29.776586,-1,148.8310,149.606
